In [18]:
import pandas as pd
import geopandas as gpd
import geojson
import json

**Read-In LOR shapes Charlottenburg-Wilmersdorf**

In [19]:
df_lor = pd.DataFrame(gpd.read_file('lor_cw.geojson'))

**Read-In LOR structural data Charlottenburg Wilmersdorf**

In [20]:
df_lor_data = pd.read_csv('EWR201812E_Matrix.csv', sep=';', dtype={'RAUMID': str})

**Merge CH shapes with CH structural data**

In [21]:
df_lor = df_lor.merge(df_lor_data[['RAUMID','E_E']], left_on='spatial_name', right_on='RAUMID')

**Select, order and rename columns**

In [22]:
df_lor = df_lor[['spatial_name','spatial_alias', 'SHAPE_AREA', 'E_E', 'geometry']]

In [23]:
df_lor.rename(columns={"spatial_name": "neighbourhood_id", 
                       "spatial_alias": "neighbourhood_name", 
                       "SHAPE_AREA": "area",
                       "E_E": "pop"
                      }, inplace=True)

In [24]:
df_lor.head(3)

,neighbourhood_id,neighbourhood_name,area,pop,geometry
0,04010101,Jungfernheide,3.066536e+06,10785,"POLYGON ((13.27033 52.54935, 13.27033 52.54922..."
1,04010102,Plötzensee,2.167532e+06,812,"POLYGON ((13.29729 52.54295, 13.29808 52.54279..."
2,04010103,Paul-Hertz-Siedlung,5.558701e+05,7881,"POLYGON ((13.29423 52.53273, 13.29481 52.53287..."


**Calculate KPI**

In [25]:
df_lor['area'] = df_lor['area']/1000000

In [26]:
df_lor['pop_den'] = df_lor['pop']/ (df_lor['area'])

In [27]:
df_lor['pop_share'] = df_lor['pop'] / df_lor['pop'].sum()

In [28]:
df_lor.head(3)

,neighbourhood_id,neighbourhood_name,area,pop,geometry,pop_den,pop_share
0,04010101,Jungfernheide,3.066536,10785,"POLYGON ((13.27033 52.54935, 13.27033 52.54922...",3516.997681,0.031597
1,04010102,Plötzensee,2.167532,812,"POLYGON ((13.29729 52.54295, 13.29808 52.54279...",374.619593,0.002379
2,04010103,Paul-Hertz-Siedlung,0.555870,7881,"POLYGON ((13.29423 52.53273, 13.29481 52.53287...",14177.773004,0.023089


**reduce to overlaps**

In [29]:
overlaps = ['04030416', 
            '04030417']

df_lor = df_lor[df_lor['neighbourhood_id'].isin(overlaps)] 

**Export to geojson**

In [30]:
def to_geojson(geometry):
    
    points = str(geometry).replace('(','').replace(')','').replace('POLYGON ','').split(', ')
    
    coordinates = list()
    
    for string in points:
        
        lat_lon = string.split(' ')
        lat_lon[0] = float(lat_lon[0])
        lat_lon[1] = float(lat_lon[1])
        
        coordinates.append(lat_lon)
        
    geojson= list([coordinates])
    return {"type": "Polygon", "coordinates": geojson}

df_lor['geojson'] = df_lor.geometry.apply(lambda x: to_geojson(x))
df_lor.head(3)

,neighbourhood_id,neighbourhood_name,area,pop,geometry,pop_den,pop_share,geojson
15,04030416,Klausenerplatz,0.478188,10652,"POLYGON ((13.28362 52.51843, 13.28360 52.51839...",22275.755905,0.031208,"{'type': 'Polygon', 'coordinates': [[[13.28361..."
16,04030417,Schloßstraße,0.562503,10976,"POLYGON ((13.28476 52.50996, 13.28538 52.51000...",19512.788265,0.032157,"{'type': 'Polygon', 'coordinates': [[[13.28476..."


In [31]:
def create_feature(x):
    
    feat = geojson.Feature(geometry=x.geojson,
                          properties={'id':x['neighbourhood_id'],
                                      'name':x['neighbourhood_name'],
                                      'area':x['area'],
                                      'pop':x['pop'],
                                      'pop_den':x['pop_den'],
                                      'pop_share':x['pop_share']})
    return feat

features = []
features = df_lor.apply(lambda row: create_feature(row), axis=1).tolist()

In [32]:
feature_collection = geojson.FeatureCollection(features)

In [33]:
feature_collection

{"features": [{"geometry": {"coordinates": [[[13.283618, 52.51843], [13.283602, 52.518387], [13.283851, 52.517827], [13.284018, 52.517457], [13.284096, 52.517282], [13.284175, 52.517119], [13.284287, 52.516934], [13.284406, 52.516777], [13.284197, 52.516691], [13.284357, 52.516496], [13.284537, 52.516263], [13.284668, 52.516043], [13.28478, 52.51582], [13.284887, 52.515538], [13.285, 52.515258], [13.285105, 52.514967], [13.285197, 52.514704], [13.285307, 52.514377], [13.285386, 52.514124], [13.285471, 52.513782], [13.285555, 52.513371], [13.285598, 52.512997], [13.28564, 52.512993], [13.286406, 52.513045], [13.28697, 52.513095], [13.28805, 52.513162], [13.28862, 52.513213], [13.288704, 52.513175], [13.290995, 52.513315], [13.291269, 52.513334], [13.294284, 52.513521], [13.294621, 52.513545], [13.294701, 52.513551], [13.296504, 52.513681], [13.296478, 52.513846], [13.296195, 52.515569], [13.296168, 52.515755], [13.296002, 52.516756], [13.295972, 52.516936], [13.295886, 52.517519], [13.2

In [34]:
with open('overlaps3.geojson', 'w', encoding='utf-8') as f:
    json.dump(feature_collection, f, ensure_ascii=False)